
## TORCH.AUTOGRAD를 사용한 자동 미분


신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파  
이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정  <br>

이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있음  
이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원  

gradient 값을 확인 하려면 requires_grad = True로 생성한 Tensor에서 .grad를 통해 값을 확인할 수 있음

In [1]:
import torch

In [2]:
# 입력 x, 매개변수 w와 b , 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망

x = torch.ones(5)   # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)   # 최적화 해야하는 매개변수
b = torch.randn(3, requires_grad=True)      # 최적화 해야하는 매개변수
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

![연산그래프](https://tutorials.pytorch.kr/_images/comp-graph.png)

w와 b는 최적화를 해야 하는 매개변수  
따라서 이러한 변수들에 대한 손실 함수의 변화도를 계산할 수 있어야 함  
이를 위해서 해당 텐서에 requires_grad 속성을 설정

연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 Function 클래스의 객체  
이 객체는 순전파 방향으로 함수를 계산하는 방법과, 역방향 전파 단계에서 도함수(derivative)를 계산하는 방법을 알고 있음  
역방향 전파 함수에 대한 참조(reference)는 텐서의 grad_fn 속성에 저장

In [3]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f25cf713d50>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f25cf713f50>


변화도(Gradient) 계산  
신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야 함  
즉, x와 y의 일부 고정값에서 
∂loss / ∂w
와
∂loss / ∂b
​가 필요  
이러한 도함수를 계산하기 위해, loss.backward() 를 호출한 다음 w.grad와 b.grad에서 값을 가져옴

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1441, 0.0443, 0.0845],
        [0.1441, 0.0443, 0.0845],
        [0.1441, 0.0443, 0.0845],
        [0.1441, 0.0443, 0.0845],
        [0.1441, 0.0443, 0.0845]])
tensor([0.1441, 0.0443, 0.0845])


변화도 추적 멈추기

In [5]:
# 방법 1.
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [7]:
# 방법 2.
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False
